In [1]:
import cv2
import imutils
import numpy as np
import glob

In [2]:
with np.load('calibration_values.npz') as calibration_values:
    mtx, dist, rvecs, tvecs = [calibration_values[i] for i in ('mtx', 'dist', 'rvecs', 'tvecs')]

In [3]:
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

In [8]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((7*5, 1, 3), np.float32) 
objp[:,:,:2] = np.mgrid[0:7,  0:5].T.reshape(-1,1,2)
print(objp[5])
print(cv2.convertPointsToHomogeneous(objp[5]))

axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

[[ 5.  0.  0.]]
[[[ 5.  0.  0.  1.]]]


In [12]:
def pixelToWorld(img_point, z, rvecs, tvecs, mtx):
    '''
    Prints world coordinate with given z value of inputted pixel coordinate.
    Input: img_point (1x2 numpy array x, y), z value of world point, rvecs, tvecs, camera mtx
    '''
    if img_point.shape != (1, 2):
        raise ValueError('Expects 1x2 numpy array as img_point, received {}'.format(img_point))
    # 3x3 rotation matrix
    rtmtx, _ = cv2.Rodrigues(rvecs)
    
    rt_tr_mtx = np.concatenate((rtmtx, tvecs), axis=1)
    transform_mtx = np.dot(mtx, rt_tr_mtx)
    u = img_point[0][0]
    v = img_point[0][1]
    desired_vec = np.array([[transform_mtx[0][2] * z + transform_mtx[0][3] - u * transform_mtx[2][2] * z - u * transform_mtx[2][3]],
                           [transform_mtx[1][2] * z + transform_mtx[1][3] - v * transform_mtx[2][2] * z - v * transform_mtx[2][3]]])
    
    desired_mat = np.array([[u * transform_mtx[2][0] - transform_mtx[0][0], u * transform_mtx[2][1] - transform_mtx[0][1]],
                           [v * transform_mtx[2][0] - transform_mtx[1][0], v * transform_mtx[2][1] - transform_mtx[1][1]]])
    inv_dmat = np.linalg.inv(desired_mat)
    pred = np.dot(inv_dmat, desired_vec)
    z_axis = np.array([[z]])
    print(pred.shape)
    print(z_axis.shape)
    pred = np.concatenate((pred, z_axis), axis=0)
    print("Prediction: {}".format(pred))

In [13]:
def getLocations(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONUP:
        img_points = np.array([[x, y]])
        pixelToWorld(img_points, 0, rvecs, tvecs, mtx)

In [15]:
cv2.namedWindow('img')
cv2.setMouseCallback('img', getLocations)

#for fname in glob.glob('checkerboard_images/*.jpg'):
for fname in ['checkerboard_images/img21.jpg']:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (7,5),None)

    if ret == True:
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
#         print(corners2)
#         print(objp)

        # Find the rotation and translation vectors.
        #_, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, None)
        _, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, dist)

        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

        img = draw(img,corners2,imgpts)
        cv2.drawChessboardCorners(img, (7,5), corners2,ret)
        cv2.imshow('img',img)
        k = cv2.waitKey(0) & 0xff
        if k == 's':
            cv2.imwrite(fname[:6]+'.png', img)

cv2.destroyAllWindows()

[[ 0.99995709  0.00874248  0.0030652 ]
 [-0.00877021  0.99991966  0.00915242]
 [-0.00298493 -0.00917891  0.99995342]]
(2, 1)
(1, 1)
Prediction: [[ 1.94923479]
 [-0.11996367]
 [ 0.        ]]
